In [1]:
import pathlib
import os

import pandas as pd

import matplotlib.pyplot as plt
import datetime
%matplotlib inline

import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import mapclassify

In [2]:
import pymysql

In [3]:
db = pymysql.connect(host='localhost',
                     user='root',
                     password='password',
                     database='chicago')
cursor = db.cursor()

In [4]:
sqlcom ="SELECT * FROM weekend_eveningPeak"
df= pd.read_sql(sqlcom, con=db)
print(df)

/opt/anaconda3/envs/python/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


            node_id         lat          lon            timestamp value_hrf
0      001e0610ee36   41.751295   -87.605288  2019/10/05 16:00:04     56.72
1      001e06113ad8   41.866786   -87.666306  2019/10/05 16:00:04     62.74
2      001e06113d20  41.8920031  -87.6116431  2019/10/05 16:00:05     56.72
3      001e0610bc10   41.736314   -87.624179  2019/10/05 16:00:06     95.61
4      001e0610ee43   41.788608   -87.598713  2019/10/05 16:00:06     56.72
...             ...         ...          ...                  ...       ...
39755  001e0610ba13   41.751238    -87.71299  2019/10/06 20:59:54     56.72
39756  001e06113d20  41.8920031  -87.6116431  2019/10/06 20:59:55     56.72
39757  001e0610ba15   41.722457    -87.57535  2019/10/06 20:59:57      55.0
39758  001e06113a24   41.788979   -87.597995  2019/10/06 20:59:57     56.72
39759  001e0610f732   41.895005   -87.745817  2019/10/06 20:59:59     56.72

[39760 rows x 5 columns]


Calculate the average sound level of each node during weekends evening peak hours

In [6]:
from decimal import Decimal
dict={}
for i in range(0,len(df)):
    key=df.iloc[i]['node_id']
    lat=df.iloc[i]['lat']
    lon=df.iloc[i]['lon']
    value=float(df.iloc[i]['value_hrf'])
    
    if key in dict:
        tempDict=dict.get(key)
        valueTemp=tempDict["value"]
        valueTemp.append(value)
        dict[key]=tempDict 
    else:
        dict[key]={"value":[value],"lon":lon,"lat":lat}

temp={}
for key in dict:
    tempDict=dict.get(key)
#     tempDict['average']=Decimal(np.mean(tempDict["value"])).quantize(Decimal('0.00'))
    tempDict['average']=np.mean(tempDict["value"])
    temp[key]=tempDict
print(temp)

{'001e0610ee36': {'value': [56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 62.74, 56.72, 56.72, 56

In [7]:
df=pd.DataFrame.from_dict(temp,orient='index')
df

,value,lon,lat,average
001e0610ee36,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55....",-87.605288,41.751295,58.121191
001e06113ad8,"[62.74, 55.0, 56.72, 56.72, 56.72, 56.72, 56.7...",-87.666306,41.866786,57.187058
001e06113d20,"[56.72, 55.0, 55.0, 56.72, 55.0, 56.72, 56.72,...",-87.6116431,41.8920031,56.926400
001e0610bc10,"[95.61, 95.61, 95.61, 95.71, 95.71, 95.61, 95....",-87.624179,41.736314,97.283270
001e0610ee43,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55....",-87.598713,41.788608,60.501252
001e06112e77,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.664343,41.786756,56.437351
001e061146cb,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55....",-87.683022,41.9140942,57.721081
001e0611462f,"[55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.7...",-87.641054,41.823527,57.128964
001e06113ace,"[62.74, 56.72, 55.0, 56.72, 55.0, 56.72, 56.72...",-87.617298,41.83107,57.265563
001e0610f703,"[62.74, 62.74, 62.74, 56.72, 56.72, 62.74, 62....",-87.67644,41.87148,59.199647


In [8]:
df.shape

(31, 4)

In [9]:
df['average']

001e0610ee36    58.121191
001e06113ad8    57.187058
001e06113d20    56.926400
001e0610bc10    97.283270
001e0610ee43    60.501252
001e06112e77    56.437351
001e061146cb    57.721081
001e0611462f    57.128964
001e06113ace    57.265563
001e0610f703    59.199647
001e06113acb    57.497969
001e061146ba    56.523272
001e0610ba13    56.387097
001e06113107    56.761831
001e061144cd    56.228571
001e06117b44    62.369480
001e061183f5    61.794522
001e0610ba46    60.705256
001e0610f6db    56.628797
001e061130f4    56.911453
001e06113a24    56.544091
001e0610ba15    56.657576
001e0610f732    61.667910
001e0610e538    57.913010
001e0611441e    58.042246
001e06118295    60.692972
001e061144be    56.498220
001e0610bbe5    56.871222
001e0610f05c    62.162930
001e0611850f    62.493783
001e061184a3    61.564742
Name: average, dtype: float64